# Práctico 1 - Parte 2

[Enunciado](https://github.com/DiploDatos/AprendizajeProfundo/blob/master/Practico.md) del trabajo práctico.

**Implementación de red neuronal [Perceptrón Multicapa](https://en.wikipedia.org/wiki/Multilayer_perceptron) (MLP).**

## Integrantes
- Mauricio Caggia
- Luciano Monforte
- Gustavo Venchiarutti
- Guillermo Robiglio

En esta segunda parte se cargan datos reducidos en la parte 1. Esto con el fin de optimizar memoria.

## Importaciones

In [16]:
import gzip
import bz2
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm, trange

## Constantes

In [18]:
ARCHIVO_SET_DE_ENTRENAMIENTO_1 = './data/training_set1.csv'
ARCHIVO_SET_DE_ENTRENAMIENTO_2 = './data/training_set2.csv'
ARCHIVO_SET_DE_ENTRENAMIENTO_3 = './data/training_set3.csv'
ARCHIVO_SET_DE_ENTRENAMIENTO_4 = './data/training_set4.csv'
ARCHIVO_SET_DE_PRUEBA = './data/test_set.csv'
ARCHIVO_SET_DE_VALIDACION = './data/validation_set.csv'
ARCHIVO_DE_EMBEDDINGS = './data/SBW-vectors-300-min5.txt.bz2'
# ARCHIVO_DE_EMBEDDINGS = '../data/glove.6B.50d.txt.gz'
EPOCHS = 1

## Carga de datos

In [4]:
%%time
file_paths = [ARCHIVO_SET_DE_ENTRENAMIENTO_1,
              ARCHIVO_SET_DE_ENTRENAMIENTO_2,
              ARCHIVO_SET_DE_ENTRENAMIENTO_3,
              ARCHIVO_SET_DE_ENTRENAMIENTO_4,
              ARCHIVO_SET_DE_PRUEBA,
              ARCHIVO_SET_DE_VALIDACION]
i = 0
df = pd.read_csv(file_paths[i])

CPU times: user 1.24 s, sys: 274 ms, total: 1.52 s
Wall time: 1.53 s


In [5]:
df.head()

,title,category
0,Casita Muñecas Barbies Pintadas,DOLLHOUSES
1,Neceser Cromado Holográfico,TOILETRY_BAGS
2,Funda Asiento A Medida D20 Chevrolet,CAR_SEAT_COVERS
3,Embrague Ford Focus One 1.8 8v Td (90cv) Desde...,AUTOMOTIVE_CLUTCH_KITS
4,Bateria Panasonic Dmwbcf10 Lumix Dmc-fx60n Dmc...,CAMERA_BATTERIES


## Construcción del Dataset

El dataset se construye a partir del dataframe de Pandas que tiene dos columnas:
- **title**
- **category**

In [6]:
class MeLiChallengeDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform
    
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, item):
        item = {
            "data": self.df.iloc[item]["title"],
            "target": self.df.iloc[item]["category"]
        }

        if self.transform:
            item = self.transform(item)
        
        return item

## Preprocesamiento de los datos

El preprocesamiento de texto tiene dos propósitos:
- Tokenizar los títulos (datos) de modo que se quiten los signos de puntuación y palabras cortas como preposiciones y conjunciones (stopwords), todas las palabras queden en minúsculas, se separen en listas de palabras, etc.
- Transformar las categorías en etiquetas numéricas.

In [7]:
class RawDataProcessor:
    def __init__(self, dataset, ignore_header=True, vocab_size=50000):
        self.filters = [lambda s: s.lower(),
                        preprocessing.strip_tags,
                        preprocessing.strip_punctuation,
                        preprocessing.strip_multiple_whitespaces,
                        preprocessing.strip_numeric,
                        preprocessing.remove_stopwords,
                        preprocessing.strip_short]
        
        # Esta clase encapsula el mapeo entre las palabras normalizadas y sus correspondientes indices 
        # https://radimrehurek.com/gensim/corpora/dictionary.html
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist()
        )
        
        # Filter the dictionary with extremos words
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.filter_extremes.html?highlight=filter_extrem
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        
        # Asigna nuevos índices a todas las palabras
        # https://tedboy.github.io/nlps/generated/generated/gensim.corpora.Dictionary.compactify.html
        # https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.compactify
        self.dictionary.compactify()
        
        # Se agregan tokens especiales
        self.dictionary.patch_with_special_tokens({"[PAD]": 0,
                                                   "[UNK]": 1})
        
        # Conversión de categorías a etiquetas
        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}


    def _preprocess_string(self, string):
        # Procesamiento de los títulos mediante la aplicación de una lista de filtros
        # Parámetro: str -> El título sin procesar
        # Salida: list -> Lista de strings
        # https://radimrehurek.com/gensim/parsing/preprocessing.html#gensim.parsing.preprocessing.preprocess_string
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        # Convierte una lista de palabras en una lista de índices
        # Parámetro: list -> Lista de palabras
        # Salida: list -> Lista de enteros (índices) en el mismo orden que las palabras
        # https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2idx
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        # Convierte un string en una lista de índices
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        # Convierte las categorías a etiquetas
        return self.target_to_idx[target]
    
    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
        
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
        
        return {
            "data": data,
            "target": target
        }

In [8]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [9]:
processor = RawDataProcessor(df)
dataset = MeLiChallengeDataset(df, transform=processor)

In [10]:
i = 1000
print(f"El dataset tiene {len(dataset)} elementos.")
print(f"Elemento #{i}:\n\tData: {dataset[i]['data']}\n\tTarget: {dataset[i]['target']}")

El dataset tiene 1200001 elementos.
Elemento #1000:
	Data: [3024, 164]
	Target: 459


## Carga del Dataset

In [11]:
pad_sequences = PadSequences()
train_loader = DataLoader(dataset,
                          batch_size=100,
                          shuffle=True,
                          collate_fn=pad_sequences,
                          drop_last=False)

## Construcción del Modelo

In [12]:
class MeLiChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with gzip.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [17]:
class MeLiChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with bz2.open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] = torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 632)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

## Algoritmo de Optimización

In [20]:
model = MeLiChallengeClassifier(ARCHIVO_DE_EMBEDDINGS, processor.dictionary, 300, True)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

## Entrenamiento del modelo

In [21]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

In [22]:
%%time
for epoch in range(EPOCHS):  # Recorre el dataset multiples veces
    model.train()
    running_loss = 0.0
    for data in train_loader:
        inputs = data['data'].to(device)
        labels = data['target'].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels.squeeze().long())
        loss.backward()
        optimizer.step()

ValueError: Expected input batch_size (1) to match target batch_size (0).

## Evaluación del Modelo

## Guardado de los parámetros del modelo